In [34]:
import requests
import random
import json

In [35]:
baseUrl = "http://localhost:8081/api/v1/"
keycloakUrl = "http://keycloak:8080"

# Institute invite

In [36]:
instituteUserId = "admin36@mailinator.com"
instituteRequestBody = {
    "instituteName": "te",
    "email": instituteUserId,
    "contactNumber": "!23"
}

In [37]:
headers = {
  'Content-Type': 'application/json'
}

response = requests.request("POST", baseUrl + "Institute/invite", headers=headers, json=instituteRequestBody)

print(response.text)
assert response.status_code == 200

{"id":"sunbird-rc.registry.invite","ver":"1.0","ets":1647406890973,"params":{"resmsgid":"","msgid":"5f29f6d2-2db7-4933-94db-9a2fbc310df9","err":"","status":"SUCCESSFUL","errmsg":""},"responseCode":"OK","result":{"Institute":{"osid":"1-74e00bc3-1f9b-40ab-bf61-be4e1d983e6f"}}}


# get institute token

In [38]:
def getToken(userId):
    headers = {
        'content-type': 'application/x-www-form-urlencoded',
    }

    data = {
      'client_id': 'registry-frontend',
      'username': userId,
      'password': 'abcd@123',
      'grant_type': 'password'
    }

    response = requests.post('%s/auth/realms/sunbird-rc/protocol/openid-connect/token'%(keycloakUrl), headers=headers, data=data)
#     print(response.json())
#     print(response.status_code)
    token = response.json()["access_token"]
    return token


In [39]:
token = getToken(instituteUserId)
print(token)

eyJhbGciOiJSUzI1NiIsInR5cCIgOiAiSldUIiwia2lkIiA6ICJGRVZ0dExTTDdYMU9pOE5BcmcxU25tN1lRZWVSODc4N3dwTDdwdzJJeE9NIn0.eyJleHAiOjE2NDc0MDc0OTgsImlhdCI6MTY0NzQwNjg5OCwianRpIjoiZDk3NGYwNjUtYzgzYy00ZTA5LThkOTItYjQyNjBmM2U3ZmY4IiwiaXNzIjoiaHR0cDovL2tleWNsb2FrOjgwODAvYXV0aC9yZWFsbXMvc3VuYmlyZC1yYyIsImF1ZCI6ImFjY291bnQiLCJzdWIiOiI5OWQ2YTI1Zi1iZTY5LTRhOWEtYjYxNy1jYTgxMTVjMDBjMDciLCJ0eXAiOiJCZWFyZXIiLCJhenAiOiJyZWdpc3RyeS1mcm9udGVuZCIsInNlc3Npb25fc3RhdGUiOiIxMDc4ZDlhYy1jOTg1LTQzOTgtOWIzNi0zMjI4YmY3NzUzMGMiLCJhY3IiOiIxIiwiYWxsb3dlZC1vcmlnaW5zIjpbImh0dHBzOi8vbG9jYWxob3N0OjQyMDIiLCJodHRwOi8vbG9jYWxob3N0OjQyMDIiLCJodHRwczovL2xvY2FsaG9zdDo0MjAwIiwiaHR0cHM6Ly9uZGVhci54aXYuaW4iLCJodHRwOi8vbG9jYWxob3N0OjQyMDAiLCJodHRwOi8vbmRlYXIueGl2LmluIiwiaHR0cDovLzIwLjE5OC42NC4xMjgiXSwicmVhbG1fYWNjZXNzIjp7InJvbGVzIjpbIm9mZmxpbmVfYWNjZXNzIiwidW1hX2F1dGhvcml6YXRpb24iLCJkZWZhdWx0LXJvbGVzLW5kZWFyIl19LCJyZXNvdXJjZV9hY2Nlc3MiOnsiYWNjb3VudCI6eyJyb2xlcyI6WyJtYW5hZ2UtYWNjb3VudCIsIm1hbmFnZS1hY2NvdW50LWxpbmtzIiwidmlldy1wcm9maWxlIl19

# get institute details

In [40]:

headers = {
  'Content-Type': 'application/json',
    'Authorization': 'bearer %s'%token,
}

response = requests.request("GET", baseUrl + "Institute", headers=headers)

print(response.text)
intituteOsid = response.json()[0]["osid"]
print(intituteOsid)
assert response.status_code == 200


[{"osUpdatedAt":"2022-03-16T05:01:32.760Z","osCreatedAt":"2022-03-16T05:01:32.760Z","osUpdatedBy":"","contactNumber":"!23","osCreatedBy":"","osid":"1-74e00bc3-1f9b-40ab-bf61-be4e1d983e6f","osOwner":["99d6a25f-be69-4a9a-b617-ca8115c00c07"],"email":"admin36@mailinator.com","instituteName":"te"}]
1-74e00bc3-1f9b-40ab-bf61-be4e1d983e6f


# Affiliate Institute

In [41]:
# add affiliation details
headers = {
    'content-type': 'application/json',
    'authorization': 'bearer %s'%token,
}

data = """{
"affiliation": [
  {
    "medium": "English",
    "board": "cbse",
    "affiliationNumber":  "123",
    "grantYear": "2000",
    "expiryYear":  "3001",
    "classes": ["Class XII"]
  }
]
}"""
# print(data)
response = requests.put('%sInstitute/%s'%(baseUrl, intituteOsid), headers=headers, data=data)
print(response)
print(response.json())
assert response.status_code == 200


# get institute details
headers = {
  'Content-Type': 'application/json',
    'Authorization': 'bearer %s'%token,
}

response = requests.request("GET", baseUrl + "Institute", headers=headers)

# print(response.text)
intituteAffiliateOsid = response.json()[0]["affiliation"][0]["osid"]
print(intituteAffiliateOsid)
assert response.status_code == 200


<Response [200]>
{'id': 'sunbird-rc.registry.update', 'ver': '1.0', 'ets': 1647406905340, 'params': {'resmsgid': '', 'msgid': 'fbeff31e-bb3e-434d-8dff-882082e5a8a1', 'err': '', 'status': 'SUCCESSFUL', 'errmsg': ''}, 'responseCode': 'OK'}
1-8d6c9916-3c83-494c-abe8-58821e2e9cdc


In [42]:
# raise attestation request
headers = {
    'content-type': 'application/json',
    'authorization': 'bearer %s'%token,
}

params = (
    ('send', 'true'),
)

data = {
    "entityName":"Institute",
    "entityId":"%s"%intituteOsid,
    "name":"instituteAffiliation",
    "propertiesOSID": {
        "affiliation": [intituteAffiliateOsid]
    }
}
response = requests.put(baseUrl+'send', headers=headers, params=params, json=data)
print(response.status_code)
response.json()
assert response.status_code == 200
print(response.json())
assert response.json()["params"]["status"] == "SUCCESSFUL"

200
{'id': 'sunbird-rc.registry.send', 'ver': '1.0', 'ets': 1647406908601, 'params': {'resmsgid': '', 'msgid': 'a346839e-580e-40c3-be96-e57a736761dd', 'err': '', 'status': 'SUCCESSFUL', 'errmsg': ''}, 'responseCode': 'OK', 'result': {'attestationOSID': '1-7dbd15cf-7adb-4d02-a64c-fdd86503eca6'}}


# cbse board attest claims

In [43]:
# get claims for cbse board
# create user manually and attach board-cbse role
boardUserId = "cbse-admin"
boardToken = getToken(boardUserId)
# print(boardToken)
headers = {
  'Content-Type': 'application/json',
    'Authorization': 'bearer %s'%boardToken,
}

response = requests.request("GET", baseUrl + "board-cbse/claims", headers=headers)
assert response.status_code == 200
print(response)

# Attesting the claim as GRANT
data = {
    "action":"GRANT_CLAIM",
}
# print(response.json())
attestingClaimId = None
for claim in response.json():
#     print(claim)
    print(claim["status"])
    if claim["status"] == "OPEN":
        attestingClaimId = claim["id"]
        print(claim["id"])
        attestResp = requests.request("POST", "%sboard-cbse/claims/%s/attest" %(baseUrl, claim["id"]), headers=headers, json=data)
        assert response.status_code == 200
        print(attestResp)
        print(attestResp.json())
        break
assert attestingClaimId != None

# intituteAffiliateOsid = response.json()[0]["affiliation"][0]["osid"]
# print(intituteAffiliateOsid)

import time
time.sleep(5)
# verify attestation status
headers = {
  'Content-Type': 'application/json',
    'Authorization': 'bearer %s'%token,
}

response = requests.request("GET", baseUrl + "Institute", headers=headers)

# print(response.text)
for affiliation in response.json()[0]["instituteAffiliation"]:
        if affiliation["_osClaimId"] == attestingClaimId:
            assert affiliation["_osState"] == "PUBLISHED"

<Response [200]>
OPEN
abf0e852-50cc-4caa-b973-db0dc4e2ebef
<Response [200]>
{'resmsgid': '', 'msgid': '8d535783-74bb-4f36-92b7-6175a2743bb4', 'err': '', 'status': 'SUCCESSFUL', 'errmsg': ''}


In [44]:
# get institute details
headers = {
  'Content-Type': 'application/json',
    'Authorization': 'bearer %s'%token,
}

response = requests.request("GET", baseUrl + "Institute", headers=headers)

print(response.text)
for affiliation in response.json()[0]["instituteAffiliation"]:
        if affiliation["_osClaimId"] == attestingClaimId:
            assert affiliation["_osState"] == "PUBLISHED"

[{"osUpdatedAt":"2022-03-16T05:02:00.003Z","osCreatedAt":"2022-03-16T05:01:32.760Z","osUpdatedBy":"","affiliation":[{"osUpdatedAt":"2022-03-16T05:02:00.003Z","grantYear":"2000","osUpdatedBy":"","classes":["Class XII"],"affiliationNumber":"123","osid":"1-8d6c9916-3c83-494c-abe8-58821e2e9cdc","medium":"English","expiryYear":"3001","board":"cbse"}],"contactNumber":"!23","osCreatedBy":"","instituteAffiliation":[{"osUpdatedAt":"2022-03-16T05:02:00.003Z","osUpdatedBy":"","_osState":"PUBLISHED","entityId":"1-74e00bc3-1f9b-40ab-bf61-be4e1d983e6f","osid":"1-7dbd15cf-7adb-4d02-a64c-fdd86503eca6","propertiesOSID":{"osid":"1-345517b9-0464-4d42-b37b-9465060881b0","osUpdatedAt":"2022-03-16T05:01:48.784Z","affiliation":["1-8d6c9916-3c83-494c-abe8-58821e2e9cdc"],"osUpdatedBy":"99d6a25f-be69-4a9a-b617-ca8115c00c07"},"propertyData":"{\"affiliation\":[{\"osUpdatedAt\":\"2022-03-16T05:01:46.265Z\",\"grantYear\":\"2000\",\"osUpdatedBy\":\"99d6a25f-be69-4a9a-b617-ca8115c00c07\",\"classes\":[\"Class XII\"],\

# Teacher attestations


In [45]:
# create new institutions
institute1 = "ABC1@mailinator.com"
institute2 = "XYZ1@mailinator.com"
instituteRequestBody = {
    "instituteName": "ABC",
    "email": institute1,
    "contactNumber": "ABC1"
}
headers = {
  'Content-Type': 'application/json'
}

response = requests.request("POST", baseUrl + "Institute/invite", headers=headers, json=instituteRequestBody)

print(response.text)
assert response.status_code == 200
institute1Osid = response.json()["result"]["Institute"]["osid"]
instituteRequestBody = {
    "instituteName": "XYZ",
    "email": institute2,
    "contactNumber": "XYZ1"
}
headers = {
  'Content-Type': 'application/json'
}

response = requests.request("POST", baseUrl + "Institute/invite", headers=headers, json=instituteRequestBody)

print(response.text)
assert response.status_code == 200
institute2Osid = response.json()["result"]["Institute"]["osid"]

{"id":"sunbird-rc.registry.invite","ver":"1.0","ets":1647406931706,"params":{"resmsgid":"","msgid":"8de9590d-335e-4476-80a8-ddea08274975","err":"","status":"SUCCESSFUL","errmsg":""},"responseCode":"OK","result":{"Institute":{"osid":"1-f8f54222-fcb0-4f94-8ceb-19c16820dff5"}}}
{"id":"sunbird-rc.registry.invite","ver":"1.0","ets":1647406931921,"params":{"resmsgid":"","msgid":"8345f6a7-e9b2-4e52-9eaa-c35faefe687b","err":"","status":"SUCCESSFUL","errmsg":""},"responseCode":"OK","result":{"Institute":{"osid":"1-416a394f-5e8a-4432-ad80-597257209e79"}}}


In [46]:
# create teacher
teacherUserId = "t7"
teacherRequestBody = {
  "identityDetails": {
    "fullName": "teacher1",
    "gender": "Male",
    "dob": "1995-10-28",
    "identityHolder": {
      "AADHAAR": "123412323"
    }
  },
  "contactDetails": {
    "email": "%s@g.com"%teacherUserId,
    "mobile": teacherUserId
  },
  "academicQualifications": [
    {
      "instituteOSID": institute1Osid,
      "instituteName": "ABC"
    }
  ],
  "experience": [
    {
      "instituteOSID": institute2Osid,
      "instituteName": "XYZ"
    }
  ]
}
headers = {
  'Content-Type': 'application/json'
}

response = requests.request("POST", baseUrl + "Teacher/invite", headers=headers, json=teacherRequestBody)

print(response.text)
assert response.status_code == 200
teacherOsid = response.json()["result"]["Teacher"]["osid"]

{"id":"sunbird-rc.registry.invite","ver":"1.0","ets":1647406935308,"params":{"resmsgid":"","msgid":"f81e7df3-d291-4a01-b5b7-0d66b5663594","err":"","status":"SUCCESSFUL","errmsg":""},"responseCode":"OK","result":{"Teacher":{"osid":"1-4475907c-5964-4724-b2b5-b60e4bf94ccc"}}}


In [47]:
teacherToken = getToken(teacherUserId)
print(teacherToken)

eyJhbGciOiJSUzI1NiIsInR5cCIgOiAiSldUIiwia2lkIiA6ICJGRVZ0dExTTDdYMU9pOE5BcmcxU25tN1lRZWVSODc4N3dwTDdwdzJJeE9NIn0.eyJleHAiOjE2NDc0MDc1MzksImlhdCI6MTY0NzQwNjkzOSwianRpIjoiMWEwNjkxNTMtMGU4My00ZTUyLWI3ZTAtNjFkYmUxYjg3MGM1IiwiaXNzIjoiaHR0cDovL2tleWNsb2FrOjgwODAvYXV0aC9yZWFsbXMvc3VuYmlyZC1yYyIsImF1ZCI6ImFjY291bnQiLCJzdWIiOiJkOTczZWVjOC0wM2FmLTRkNmUtODVhZC02ZDI3ZWY0ODYxOTkiLCJ0eXAiOiJCZWFyZXIiLCJhenAiOiJyZWdpc3RyeS1mcm9udGVuZCIsInNlc3Npb25fc3RhdGUiOiI5NWJmYzY1OS1jZGUzLTQyM2MtYjVmOS0zNTFlYmMyZGRjZTciLCJhY3IiOiIxIiwiYWxsb3dlZC1vcmlnaW5zIjpbImh0dHBzOi8vbG9jYWxob3N0OjQyMDIiLCJodHRwOi8vbG9jYWxob3N0OjQyMDIiLCJodHRwczovL2xvY2FsaG9zdDo0MjAwIiwiaHR0cHM6Ly9uZGVhci54aXYuaW4iLCJodHRwOi8vbG9jYWxob3N0OjQyMDAiLCJodHRwOi8vbmRlYXIueGl2LmluIiwiaHR0cDovLzIwLjE5OC42NC4xMjgiXSwicmVhbG1fYWNjZXNzIjp7InJvbGVzIjpbIm9mZmxpbmVfYWNjZXNzIiwidW1hX2F1dGhvcml6YXRpb24iLCJkZWZhdWx0LXJvbGVzLW5kZWFyIl19LCJyZXNvdXJjZV9hY2Nlc3MiOnsiYWNjb3VudCI6eyJyb2xlcyI6WyJtYW5hZ2UtYWNjb3VudCIsIm1hbmFnZS1hY2NvdW50LWxpbmtzIiwidmlldy1wcm9maWxlIl19

In [48]:
# get teacher details
headers = {
  'Content-Type': 'application/json',
    'Authorization': 'bearer %s'%teacherToken,
}

response = requests.request("GET", baseUrl + "Teacher", headers=headers)

print(response.text)
assert response.status_code == 200
teacherEntity = response.json()[0]
print(teacherEntity["experience"][0]["osid"])
experienceOsid = teacherEntity["experience"][0]["osid"]
print(teacherEntity["academicQualifications"][0]["osid"])
qualificationOsid = teacherEntity["academicQualifications"][0]["osid"]

[{"osUpdatedAt":"2022-03-16T05:02:15.601Z","academicQualifications":[{"osUpdatedAt":"2022-03-16T05:02:15.601Z","osCreatedAt":"2022-03-16T05:02:15.601Z","osUpdatedBy":"","osCreatedBy":"","osid":"1-c15c6b3f-456d-4087-aea1-f27f306f0c0b","instituteOSID":"1-f8f54222-fcb0-4f94-8ceb-19c16820dff5","instituteName":"ABC"}],"osCreatedAt":"2022-03-16T05:02:15.601Z","osUpdatedBy":"","contactDetails":{"osid":"1-72afc1a9-6a6e-4042-9700-fb56711b54a7","osUpdatedAt":"2022-03-16T05:02:15.601Z","osCreatedAt":"2022-03-16T05:02:15.601Z","osUpdatedBy":"","mobile":"t7","osCreatedBy":"","email":"t7@g.com"},"osCreatedBy":"","experience":[{"osUpdatedAt":"2022-03-16T05:02:15.601Z","osCreatedAt":"2022-03-16T05:02:15.601Z","osUpdatedBy":"","osCreatedBy":"","osid":"1-ef3b6d5c-d459-447a-a5f6-35f62d391a5c","instituteOSID":"1-416a394f-5e8a-4432-ad80-597257209e79","instituteName":"XYZ"}],"osid":"1-4475907c-5964-4724-b2b5-b60e4bf94ccc","identityDetails":{"osid":"1-e68c52cb-6cef-48c3-a49d-c1ffb9f19628","identityHolder":{"

# raise teacher's education attestation

In [49]:
# raise attestation request
headers = {
    'content-type': 'application/json',
    'authorization': 'bearer %s'%teacherToken,
}

data = {
    "entityName":"Teacher",
    "entityId":"%s"%teacherOsid,
    "name":"teacherExperience",
    "propertiesOSID": {
        "experience": [experienceOsid]
    }
}
response = requests.put(baseUrl+'send', headers=headers, json=data)
print(response.status_code)
response.json()
assert response.status_code == 200
print(response.json())
assert response.json()["params"]["status"] == "SUCCESSFUL"

200
{'id': 'sunbird-rc.registry.send', 'ver': '1.0', 'ets': 1647406955903, 'params': {'resmsgid': '', 'msgid': '305aadb6-acc6-47d9-a335-61c388a7975b', 'err': '', 'status': 'SUCCESSFUL', 'errmsg': ''}, 'responseCode': 'OK', 'result': {'attestationOSID': '1-6888f611-cf80-45d8-832e-572f18194d42'}}


In [50]:
# get institute token
institute1Token = getToken(institute1)
print(institute1Token)
institute2Token = getToken(institute2)
print(institute2Token)

eyJhbGciOiJSUzI1NiIsInR5cCIgOiAiSldUIiwia2lkIiA6ICJGRVZ0dExTTDdYMU9pOE5BcmcxU25tN1lRZWVSODc4N3dwTDdwdzJJeE9NIn0.eyJleHAiOjE2NDc0MDc1NTcsImlhdCI6MTY0NzQwNjk1NywianRpIjoiZDc0ZWNjYTQtNzY5ZC00NjBhLWFiZjctMjc3NmQ3OTk0NjgwIiwiaXNzIjoiaHR0cDovL2tleWNsb2FrOjgwODAvYXV0aC9yZWFsbXMvc3VuYmlyZC1yYyIsImF1ZCI6ImFjY291bnQiLCJzdWIiOiI3N2VhOWU0Zi1hOGFmLTRlMjItOGY1OS02NWVmYTFkNWZjYmQiLCJ0eXAiOiJCZWFyZXIiLCJhenAiOiJyZWdpc3RyeS1mcm9udGVuZCIsInNlc3Npb25fc3RhdGUiOiJiMWE2NGQzYy02MGI2LTQxZjEtYWIzZS1jZTc0YTY1Njc0MDgiLCJhY3IiOiIxIiwiYWxsb3dlZC1vcmlnaW5zIjpbImh0dHBzOi8vbG9jYWxob3N0OjQyMDIiLCJodHRwOi8vbG9jYWxob3N0OjQyMDIiLCJodHRwczovL2xvY2FsaG9zdDo0MjAwIiwiaHR0cHM6Ly9uZGVhci54aXYuaW4iLCJodHRwOi8vbG9jYWxob3N0OjQyMDAiLCJodHRwOi8vbmRlYXIueGl2LmluIiwiaHR0cDovLzIwLjE5OC42NC4xMjgiXSwicmVhbG1fYWNjZXNzIjp7InJvbGVzIjpbIm9mZmxpbmVfYWNjZXNzIiwidW1hX2F1dGhvcml6YXRpb24iLCJkZWZhdWx0LXJvbGVzLW5kZWFyIl19LCJyZXNvdXJjZV9hY2Nlc3MiOnsiYWNjb3VudCI6eyJyb2xlcyI6WyJtYW5hZ2UtYWNjb3VudCIsIm1hbmFnZS1hY2NvdW50LWxpbmtzIiwidmlldy1wcm9maWxlIl19

In [51]:
# get institute claims and grant the claim
headers = {
  'Content-Type': 'application/json',
    'Authorization': 'bearer %s'%institute2Token,
}

response = requests.request("GET", baseUrl + "Institute/claims", headers=headers)
assert response.status_code == 200
print(response)

# Attesting the claim as GRANT
data = {
    "action":"GRANT_CLAIM",
}
print(response.json())
attestingClaimId = None
for claim in response.json():
#     print(claim)
    print(claim["status"])
    if claim["status"] == "OPEN":
        attestingClaimId = claim["id"]
        print(claim["id"])
        attestResp = requests.request("POST", "%sInstitute/claims/%s/attest" %(baseUrl, claim["id"]), headers=headers, json=data)
        assert response.status_code == 200
        print(attestResp)
        print(attestResp.json())
        break
assert attestingClaimId != None


<Response [200]>
[{'id': '010e75d5-43f8-4efe-aaa5-f3d4e8850610', 'entity': 'Teacher', 'entityId': '1-4475907c-5964-4724-b2b5-b60e4bf94ccc', 'propertyURI': '', 'createdAt': '2022-03-16T05:02:36.610+00:00', 'attestedOn': None, 'status': 'OPEN', 'conditions': "(ATTESTOR#$.osid#.contains('1-416a394f-5e8a-4432-ad80-597257209e79'))", 'attestorEntity': 'Institute', 'requestorName': None, 'propertyData': '{"experience":[{"osUpdatedAt":"2022-03-16T05:02:15.601Z","osCreatedAt":"2022-03-16T05:02:15.601Z","osUpdatedBy":"","osCreatedBy":"","osid":"1-ef3b6d5c-d459-447a-a5f6-35f62d391a5c","instituteOSID":"1-416a394f-5e8a-4432-ad80-597257209e79","instituteName":"XYZ"}]}', 'attestationId': '1-6888f611-cf80-45d8-832e-572f18194d42', 'attestationName': 'teacherExperience', 'closed': False}]
OPEN
010e75d5-43f8-4efe-aaa5-f3d4e8850610
<Response [200]>
{'resmsgid': '', 'msgid': '77c298ac-0bf5-456e-82e8-8457862e6030', 'err': '', 'status': 'SUCCESSFUL', 'errmsg': ''}


In [52]:
import time
time.sleep(5)
# verify attestation status
headers = {
  'Content-Type': 'application/json',
    'Authorization': 'bearer %s'%teacherToken,
}

response = requests.request("GET", baseUrl + "Teacher", headers=headers)

# print(response.text)
for affiliation in response.json()[0]["teacherExperience"]:
        if affiliation["_osClaimId"] == attestingClaimId:
            assert affiliation["_osState"] == "PUBLISHED"


In [53]:
# raise qualification attestation request
headers = {
    'content-type': 'application/json',
    'authorization': 'bearer %s'%teacherToken,
}

data = {
    "entityName":"Teacher",
    "entityId":"%s"%teacherOsid,
    "name":"teacherAcademicsQualification",
    "propertiesOSID": {
        "academicQualifications": [qualificationOsid]
    }
}
response = requests.put(baseUrl+'send', headers=headers, json=data)
print(response.status_code)
response.json()
assert response.status_code == 200
print(response.json())
assert response.json()["params"]["status"] == "SUCCESSFUL"



200
{'id': 'sunbird-rc.registry.send', 'ver': '1.0', 'ets': 1647406968481, 'params': {'resmsgid': '', 'msgid': '83db1b59-148b-48f8-a7f0-1e6c3c831f06', 'err': '', 'status': 'SUCCESSFUL', 'errmsg': ''}, 'responseCode': 'OK', 'result': {'attestationOSID': '1-feb35fd7-da36-43ab-b2e5-168a702c3553'}}


In [54]:

# get institute token
institute1Token = getToken(institute1)
print(institute1Token)

# get institute claims and grant the claim
headers = {
  'Content-Type': 'application/json',
    'Authorization': 'bearer %s'%institute1Token,
}

response = requests.request("GET", baseUrl + "Institute/claims", headers=headers)
assert response.status_code == 200
print(response)

# Attesting the claim as GRANT
data = {
    "action":"GRANT_CLAIM",
}
print(response.json())
attestingClaimId = None
for claim in response.json():
#     print(claim)
    print(claim["status"])
    if claim["status"] == "OPEN":
        attestingClaimId = claim["id"]
        print(claim["id"])
        attestResp = requests.request("POST", "%sInstitute/claims/%s/attest" %(baseUrl, claim["id"]), headers=headers, json=data)
        assert response.status_code == 200
        print(attestResp)
        print(attestResp.json())
        break
assert attestingClaimId != None

eyJhbGciOiJSUzI1NiIsInR5cCIgOiAiSldUIiwia2lkIiA6ICJGRVZ0dExTTDdYMU9pOE5BcmcxU25tN1lRZWVSODc4N3dwTDdwdzJJeE9NIn0.eyJleHAiOjE2NDc0MDc1NzIsImlhdCI6MTY0NzQwNjk3MiwianRpIjoiZmZhN2Y5YTMtMjA3ZS00MzY2LWIwZDMtYTkxOGFhMzUxOGJlIiwiaXNzIjoiaHR0cDovL2tleWNsb2FrOjgwODAvYXV0aC9yZWFsbXMvc3VuYmlyZC1yYyIsImF1ZCI6ImFjY291bnQiLCJzdWIiOiI3N2VhOWU0Zi1hOGFmLTRlMjItOGY1OS02NWVmYTFkNWZjYmQiLCJ0eXAiOiJCZWFyZXIiLCJhenAiOiJyZWdpc3RyeS1mcm9udGVuZCIsInNlc3Npb25fc3RhdGUiOiIwZjMxNTk5NS03YWYwLTQyNDMtYjcwZC1lOTE1YzU0OGIzYzgiLCJhY3IiOiIxIiwiYWxsb3dlZC1vcmlnaW5zIjpbImh0dHBzOi8vbG9jYWxob3N0OjQyMDIiLCJodHRwOi8vbG9jYWxob3N0OjQyMDIiLCJodHRwczovL2xvY2FsaG9zdDo0MjAwIiwiaHR0cHM6Ly9uZGVhci54aXYuaW4iLCJodHRwOi8vbG9jYWxob3N0OjQyMDAiLCJodHRwOi8vbmRlYXIueGl2LmluIiwiaHR0cDovLzIwLjE5OC42NC4xMjgiXSwicmVhbG1fYWNjZXNzIjp7InJvbGVzIjpbIm9mZmxpbmVfYWNjZXNzIiwidW1hX2F1dGhvcml6YXRpb24iLCJkZWZhdWx0LXJvbGVzLW5kZWFyIl19LCJyZXNvdXJjZV9hY2Nlc3MiOnsiYWNjb3VudCI6eyJyb2xlcyI6WyJtYW5hZ2UtYWNjb3VudCIsIm1hbmFnZS1hY2NvdW50LWxpbmtzIiwidmlldy1wcm9maWxlIl19

In [55]:

import time
time.sleep(3)
# verify attestation status
headers = {
  'Content-Type': 'application/json',
    'Authorization': 'bearer %s'%teacherToken,
}

response = requests.request("GET", baseUrl + "Teacher", headers=headers)


for affiliation in response.json()[0]["teacherAcademicsQualification"]:
    print(affiliation)
    if affiliation["_osClaimId"] == attestingClaimId:
        assert affiliation["_osState"] == "PUBLISHED"

{'osUpdatedAt': '2022-03-16T05:02:52.429Z', 'osUpdatedBy': '', '_osState': 'PUBLISHED', 'entityId': '1-4475907c-5964-4724-b2b5-b60e4bf94ccc', 'osid': '1-feb35fd7-da36-43ab-b2e5-168a702c3553', 'propertiesOSID': {'osid': '1-88bd050f-d83e-4c2b-bfec-f662dcec8a92', 'osUpdatedAt': '2022-03-16T05:02:48.543Z', 'osUpdatedBy': 'd973eec8-03af-4d6e-85ad-6d27ef486199', 'academicQualifications': ['1-c15c6b3f-456d-4087-aea1-f27f306f0c0b']}, 'propertyData': '{"academicQualifications":[{"osUpdatedAt":"2022-03-16T05:02:40.022Z","osCreatedAt":"2022-03-16T05:02:15.601Z","osUpdatedBy":"","osCreatedBy":"","osid":"1-c15c6b3f-456d-4087-aea1-f27f306f0c0b","instituteOSID":"1-f8f54222-fcb0-4f94-8ceb-19c16820dff5","instituteName":"ABC"}]}', 'entityName': 'Teacher', 'name': 'teacherAcademicsQualification', '_osAttestedData': '{"academicQualifications":[{"osUpdatedAt":"2022-03-16T05:02:40.022Z","osCreatedAt":"2022-03-16T05:02:15.601Z","osUpdatedBy":"","osCreatedBy":"","osid":"1-c15c6b3f-456d-4087-aea1-f27f306f0c0b"

# Student Attestations

In [61]:
# create teacher
studentUserId = "s5"
studentRequestBody = {
  "identityDetails": {
    "fullName": "student1",
    "gender": "Male",
    "dob": "1995-10-28",
    "identityHolder": {
      "AADHAAR": "123412323"
    }
  },
  "contactDetails": {
    "email": "%s@g.com"%studentUserId,
    "mobile": studentUserId
  },
  "educationDetails": [
    {
      "instituteOSID": institute2Osid,
      "instituteName": "XYZ"
    }
  ]
}
headers = {
  'Content-Type': 'application/json'
}

response = requests.request("POST", baseUrl + "Student/invite", headers=headers, json=studentRequestBody)

print(response.text)
assert response.status_code == 200
studentOsid = response.json()["result"]["Student"]["osid"]

{"id":"sunbird-rc.registry.invite","ver":"1.0","ets":1647407039305,"params":{"resmsgid":"","msgid":"363d9d30-f76c-4c4c-b94c-8f43e478f6bd","err":"","status":"SUCCESSFUL","errmsg":""},"responseCode":"OK","result":{"Student":{"osid":"1-f89b0b11-9032-409a-8534-9ff344d06eca"}}}


In [62]:
studentToken = getToken(studentUserId)
print(studentToken)

eyJhbGciOiJSUzI1NiIsInR5cCIgOiAiSldUIiwia2lkIiA6ICJGRVZ0dExTTDdYMU9pOE5BcmcxU25tN1lRZWVSODc4N3dwTDdwdzJJeE9NIn0.eyJleHAiOjE2NDc0MDc2NDAsImlhdCI6MTY0NzQwNzA0MCwianRpIjoiMWRlNDE1MDQtYThiMi00YWYxLWEzYzgtODUyZTlmNTZkZjA1IiwiaXNzIjoiaHR0cDovL2tleWNsb2FrOjgwODAvYXV0aC9yZWFsbXMvc3VuYmlyZC1yYyIsImF1ZCI6ImFjY291bnQiLCJzdWIiOiIzN2QwMmM2ZS1jNzllLTQ0YTktYWMzZi03YmNlY2Y5MjBmZTkiLCJ0eXAiOiJCZWFyZXIiLCJhenAiOiJyZWdpc3RyeS1mcm9udGVuZCIsInNlc3Npb25fc3RhdGUiOiI1YzBlNWUyNS1hMDIwLTRiODctYTk4MS00ODdkYzNlYTg4NDciLCJhY3IiOiIxIiwiYWxsb3dlZC1vcmlnaW5zIjpbImh0dHBzOi8vbG9jYWxob3N0OjQyMDIiLCJodHRwOi8vbG9jYWxob3N0OjQyMDIiLCJodHRwczovL2xvY2FsaG9zdDo0MjAwIiwiaHR0cHM6Ly9uZGVhci54aXYuaW4iLCJodHRwOi8vbG9jYWxob3N0OjQyMDAiLCJodHRwOi8vbmRlYXIueGl2LmluIiwiaHR0cDovLzIwLjE5OC42NC4xMjgiXSwicmVhbG1fYWNjZXNzIjp7InJvbGVzIjpbIm9mZmxpbmVfYWNjZXNzIiwidW1hX2F1dGhvcml6YXRpb24iLCJkZWZhdWx0LXJvbGVzLW5kZWFyIl19LCJyZXNvdXJjZV9hY2Nlc3MiOnsiYWNjb3VudCI6eyJyb2xlcyI6WyJtYW5hZ2UtYWNjb3VudCIsIm1hbmFnZS1hY2NvdW50LWxpbmtzIiwidmlldy1wcm9maWxlIl19

In [63]:
# get student details
headers = {
  'Content-Type': 'application/json',
    'Authorization': 'bearer %s'%studentToken,
}

response = requests.request("GET", baseUrl + "Student", headers=headers)

print(response.text)
assert response.status_code == 200
print(response.json()[0]["educationDetails"][0]["osid"])
educationDetailsOsid = response.json()[0]["educationDetails"][0]["osid"]

[{"educationDetails":[{"osid":"1-672eef11-c0e3-4c09-a599-3d628d0be8d9","instituteOSID":"1-416a394f-5e8a-4432-ad80-597257209e79","instituteName":"XYZ"}],"contactDetails":{"osid":"1-39a0faca-d612-4f2a-a5c9-4089e380e108","mobile":"s5","email":"s5@g.com"},"osid":"1-f89b0b11-9032-409a-8534-9ff344d06eca","identityDetails":{"osid":"1-4238990a-bf61-462c-841a-6df5f6c38dcc","identityHolder":{"osid":"1-f592308c-90e3-4755-9664-51f5709a0b7a","AADHAAR":"123412323"},"gender":"Male","dob":"1995-10-28","fullName":"student1"},"osOwner":["37d02c6e-c79e-44a9-ac3f-7bcecf920fe9"]}]
1-672eef11-c0e3-4c09-a599-3d628d0be8d9


In [64]:
# raise qualification attestation request
headers = {
    'content-type': 'application/json',
    'authorization': 'bearer %s'%studentToken,
}

data = {
    "entityName":"Student",
    "entityId":"%s"%studentOsid,
    "name":"studentInstituteAttest",
    "propertiesOSID": {
        "educationDetails": [educationDetailsOsid]
    }
}
response = requests.put(baseUrl+'send', headers=headers, json=data)
print(response.status_code)
response.json()
assert response.status_code == 200
print(response.json())
assert response.json()["params"]["status"] == "SUCCESSFUL"


200
{'id': 'sunbird-rc.registry.send', 'ver': '1.0', 'ets': 1647407043398, 'params': {'resmsgid': '', 'msgid': 'ebe47837-98b3-44b4-9c92-81c9f025f43f', 'err': '', 'status': 'SUCCESSFUL', 'errmsg': ''}, 'responseCode': 'OK', 'result': {'attestationOSID': '1-85f67859-685b-4380-8379-3f8dad1dd17f'}}


In [65]:
# get teacher token
teacherToken = getToken(teacherUserId)
print(teacherToken)

# get institute claims and grant the claim
headers = {
  'Content-Type': 'application/json',
    'Authorization': 'bearer %s'%teacherToken,
}

response = requests.request("GET", baseUrl + "Teacher/claims", headers=headers)
assert response.status_code == 200
print(response)

# Attesting the claim as GRANT
data = {
    "action":"GRANT_CLAIM",
}
print(response.json())
attestingClaimId = None
for claim in response.json():
#     print(claim)
    print(claim["status"])
    if claim["status"] == "OPEN":
        attestingClaimId = claim["id"]
        print(claim["id"])
        attestResp = requests.request("POST", "%sTeacher/claims/%s/attest" %(baseUrl, claim["id"]), headers=headers, json=data)
        assert response.status_code == 200
        print(attestResp)
        print(attestResp.json())
        break
assert attestingClaimId != None

eyJhbGciOiJSUzI1NiIsInR5cCIgOiAiSldUIiwia2lkIiA6ICJGRVZ0dExTTDdYMU9pOE5BcmcxU25tN1lRZWVSODc4N3dwTDdwdzJJeE9NIn0.eyJleHAiOjE2NDc0MDc2NDYsImlhdCI6MTY0NzQwNzA0NiwianRpIjoiNTg4M2EzNjYtM2E4My00YmI4LTk5Y2QtY2FlODdjOWQxNDExIiwiaXNzIjoiaHR0cDovL2tleWNsb2FrOjgwODAvYXV0aC9yZWFsbXMvc3VuYmlyZC1yYyIsImF1ZCI6ImFjY291bnQiLCJzdWIiOiJkOTczZWVjOC0wM2FmLTRkNmUtODVhZC02ZDI3ZWY0ODYxOTkiLCJ0eXAiOiJCZWFyZXIiLCJhenAiOiJyZWdpc3RyeS1mcm9udGVuZCIsInNlc3Npb25fc3RhdGUiOiJmNDNiZTQxOS1jYWI3LTQ5YzYtYTdiOS0xN2ZkMzllMzMxMTgiLCJhY3IiOiIxIiwiYWxsb3dlZC1vcmlnaW5zIjpbImh0dHBzOi8vbG9jYWxob3N0OjQyMDIiLCJodHRwOi8vbG9jYWxob3N0OjQyMDIiLCJodHRwczovL2xvY2FsaG9zdDo0MjAwIiwiaHR0cHM6Ly9uZGVhci54aXYuaW4iLCJodHRwOi8vbG9jYWxob3N0OjQyMDAiLCJodHRwOi8vbmRlYXIueGl2LmluIiwiaHR0cDovLzIwLjE5OC42NC4xMjgiXSwicmVhbG1fYWNjZXNzIjp7InJvbGVzIjpbIm9mZmxpbmVfYWNjZXNzIiwidW1hX2F1dGhvcml6YXRpb24iLCJkZWZhdWx0LXJvbGVzLW5kZWFyIl19LCJyZXNvdXJjZV9hY2Nlc3MiOnsiYWNjb3VudCI6eyJyb2xlcyI6WyJtYW5hZ2UtYWNjb3VudCIsIm1hbmFnZS1hY2NvdW50LWxpbmtzIiwidmlldy1wcm9maWxlIl19

In [66]:

import time
time.sleep(1)
# verify attestation status
headers = {
  'Content-Type': 'application/json',
    'Authorization': 'bearer %s'%studentToken,
}

response = requests.request("GET", baseUrl + "Student", headers=headers)


for affiliation in response.json()[0]["studentInstituteAttest"]:
    print(affiliation)
    if affiliation["_osClaimId"] == attestingClaimId:
        assert affiliation["_osState"] == "PUBLISHED"

{'entityName': 'Student', '_osState': 'PUBLISHED', 'name': 'studentInstituteAttest', 'entityId': '1-f89b0b11-9032-409a-8534-9ff344d06eca', 'osid': '1-85f67859-685b-4380-8379-3f8dad1dd17f', '_osAttestedData': '{"name":"student1","educationDetails":[{"osid":"1-672eef11-c0e3-4c09-a599-3d628d0be8d9","instituteOSID":"1-416a394f-5e8a-4432-ad80-597257209e79","instituteName":"XYZ"}]}', 'propertiesOSID': {'osid': '1-bf5121c6-754c-4ddd-aaad-3455bab06d70', 'educationDetails': ['1-672eef11-c0e3-4c09-a599-3d628d0be8d9']}, '_osClaimId': '3bcff842-1c8a-45dc-b42a-78655fd0a8dc', 'propertyData': '{"name":"student1","educationDetails":[{"osid":"1-672eef11-c0e3-4c09-a599-3d628d0be8d9","instituteOSID":"1-416a394f-5e8a-4432-ad80-597257209e79","instituteName":"XYZ"}]}'}
